In [1]:
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation

from explaneat.core.neuralneat import NeuralNeat as NeuralNeat

import random

import torch
import torch.nn as nn
import torch.optim as optim

import neat

from explaneat.core.utility import MethodTimer


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
# from explaneat.visualization import visualize
from explaneat.core.experiment import ExperimentReporter

from explaneat.core.neuralneat import NeuralNeat as nneat

import torch.optim as optim

from torch.functional import F








In [2]:
RANDOM_SEED      = 42
torch.manual_seed(RANDOM_SEED)


USE_CUDA = torch.cuda.is_available()
USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")


In [3]:

def xor(a, b, threshold = 0.5):
    response = False
    if a > threshold and b < threshold:
        response = True
    if a < threshold and b > threshold:
        response = True
    # return (1.0, 0.0) if response else (0.0, 1.0)
    return 1.0 if response else 0.0
    

def create_n_points(n, size, min=0.0, max=1.0):
    data = []
    for _ in range(n):
        data.append([
            random.uniform(min, max) for ii in range(size)
        ])

    return data

# correct solution:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

def overUnder(val, threshold):
    return 1. if val > threshold else 0

xor_inputs = create_n_points(400, 2, -1, 1)

xor_outputs = [
    [xor(tup[0], tup[1], 0)] for tup in xor_inputs
]

In [4]:
config_path = "./config_xor"
base_config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)

def instantiate_population(config, xs, ys, saveLocation):

    if not os.path.exists(saveLocation):
        os.makedirs(saveLocation)
        
    config.save(os.path.join(saveLocation, 'config.conf'))

    # Create the population, which is the top-level object for a NEAT run.
    with MethodTimer("Backprop everything"):
        p = BackpropPopulation(config, 
                                xs, 
                                ys, 
                                criterion=nn.BCEWithLogitsLoss())

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5, filename_prefix=str(saveLocation) + "checkpoint-" ))
    bpReporter = backprop.BackpropReporter(True)
    p.add_reporter(bpReporter)
    p.add_reporter(ExperimentReporter(saveLocation))
    
    return p

def eval_genomes(genomes, config):
    
    print(genomes)
    loss = nn.BCELoss()
    loss = loss.to(device)
    for genome_id, genome in genomes.items():
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in xor_inputs:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(xor_outputs)))

config = base_config
saveLocation = './'
maxNGenerations = 3
p = instantiate_population(config, xor_inputs, xor_outputs, saveLocation)
# Run for up to nGenerations generations.
winner = p.run(eval_genomes, maxNGenerations, nEpochs = 25)

g = p.best_genome


The function - Backprop everything - has just started at 1628760405.9732249
The function - Backprop everything - took 0.0012791156768798828 seconds to complete
The function - generationStart - has just started at 1628760405.974653

 ****** Running generation 0 ****** 

The function - generationStart - took 4.00543212890625e-05 seconds to complete
The function - pre_backprop - has just started at 1628760405.974713
The function - pre_backprop - took 0.00023674964904785156 seconds to complete
The function - backprop - has just started at 1628760405.975019
about to start backprop with 25 epochs
mean improvement: -0.006466143845876071
best improvement: -0.007149703045657874
best loss: 0.5227470527615088
The function - backprop - took 0.039856910705566406 seconds to complete
The function - post_backprop - has just started at 1628760406.0148952
The function - post_backprop - took 2.8848648071289062e-05 seconds to complete
The function - evaluate fitness - has just started at 1628760406.014987

In [5]:
print(g)

Key: 8
Fitness: 0.8307158946990967
Nodes:
	0 DefaultNodeGene(key=0, bias=0.4476301109540974, response=1.0, activation=sigmoid, aggregation=sum)
	1 DefaultNodeGene(key=1, bias=-0.5117156753882777, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-2, 0), weight=2.0754810036626328, enabled=False)
	DefaultConnectionGene(key=(-2, 1), weight=1.3891605078468516, enabled=True)
	DefaultConnectionGene(key=(-1, 0), weight=0.38695528738017726, enabled=True)
	DefaultConnectionGene(key=(1, 0), weight=-0.2843673573992772, enabled=True)


In [6]:
net = nneat(g, p.config, criterion=nn.BCEWithLogitsLoss())


In [7]:
net.genome.connections
# Error is in inputs not being updated with extra info from skip layers

{(-1, 0): <neat.genes.DefaultConnectionGene at 0x7ffd20d7cc70>,
 (-2, 0): <neat.genes.DefaultConnectionGene at 0x7ffd20d7c310>,
 (-2, 1): <neat.genes.DefaultConnectionGene at 0x7ffd20da0b50>,
 (1, 0): <neat.genes.DefaultConnectionGene at 0x7ffd20da0bb0>}

In [11]:
net.forward(torch.tensor( xor_inputs[:10], dtype=torch.float64) ) [:10]

tensor([[0.5457],
        [0.6488],
        [0.6492],
        [0.5075],
        [0.5242],
        [0.5502],
        [0.6632],
        [0.6174],
        [0.5239],
        [0.5823]], dtype=torch.float64, grad_fn=<SliceBackward>)

In [12]:
net.forward(torch.tensor( xor_inputs, dtype=torch.float64) )[:20]

tensor([[0.5457],
        [0.6488],
        [0.6492],
        [0.5075],
        [0.5242],
        [0.5502],
        [0.6632],
        [0.6174],
        [0.5239],
        [0.5823],
        [0.6221],
        [0.6371],
        [0.5381],
        [0.6160],
        [0.6312],
        [0.6010],
        [0.5357],
        [0.6445],
        [0.6276],
        [0.6127]], dtype=torch.float64, grad_fn=<SliceBackward>)

In [10]:
# net.optimiser.zero_grad()

In [11]:
# net.optimise(xor_inputs, xor_outputs)

In [12]:
optimiser = optim.Adadelta(net.parameters())

In [13]:
net.weights

{1: Parameter containing:
 tensor([[-0.6117],
         [-0.6330]], dtype=torch.float64, requires_grad=True),
 0: Parameter containing:
 tensor([], size=(0, 2), dtype=torch.float64, requires_grad=True)}

In [14]:
optimizer = optimiser
losses = []
for i in range(10000):
    preds = net.forward(torch.tensor( xor_inputs, dtype=torch.float64) )
    loss = F.mse_loss(preds, torch.tensor( xor_outputs, dtype=torch.float64)).sqrt()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    losses.append(loss)  
losses[-10:]


[tensor(0.4980, dtype=torch.float64, grad_fn=<SqrtBackward>),
 tensor(0.4980, dtype=torch.float64, grad_fn=<SqrtBackward>),
 tensor(0.4980, dtype=torch.float64, grad_fn=<SqrtBackward>),
 tensor(0.4980, dtype=torch.float64, grad_fn=<SqrtBackward>),
 tensor(0.4980, dtype=torch.float64, grad_fn=<SqrtBackward>),
 tensor(0.4980, dtype=torch.float64, grad_fn=<SqrtBackward>),
 tensor(0.4980, dtype=torch.float64, grad_fn=<SqrtBackward>),
 tensor(0.4980, dtype=torch.float64, grad_fn=<SqrtBackward>),
 tensor(0.4980, dtype=torch.float64, grad_fn=<SqrtBackward>),
 tensor(0.4980, dtype=torch.float64, grad_fn=<SqrtBackward>)]

In [15]:
preds[:10]

tensor([[0.4630],
        [0.5585],
        [0.4334],
        [0.5061],
        [0.5118],
        [0.5589],
        [0.4330],
        [0.4321],
        [0.5393],
        [0.5262]], dtype=torch.float64, grad_fn=<SliceBackward>)

In [16]:
net.weights

{1: Parameter containing:
 tensor([[0.0062],
         [0.3170]], dtype=torch.float64, requires_grad=True),
 0: Parameter containing:
 tensor([], size=(0, 2), dtype=torch.float64, requires_grad=True)}